In [1]:
from bs4 import BeautifulSoup
import requests


In [2]:
# ホームから各チームの「選手一覧」のURLを取得する
url = "https://baseball-data.com/"
headers = {"User-Agent": "Mozilla/5.0"}  # リクエストヘッダーがないとbot判定され403が返される


def get_players_list_urls(url, headers):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    table = soup.find_all("table", attrs={"class": "stats-menu"})
    tables = table[:2]
    team_urls = []
    for table in tables:
        trs = table.find_all("tr")
        for tr in trs:
            tds = tr.find_all("td")
            team_url = tds[1].a.get("href")
            team_urls.append(team_url)
    return team_urls


team_urls = get_players_list_urls(url, headers)

In [3]:
# 選手一覧から各選手のURLを取得する
def get_player_urls(url, headers):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    trs = soup.tbody.find_all("tr")
    player_urls = []
    for tr in trs:
        tds = tr.find_all("td")
        player_url = tds[1].a.get("href")
        player_urls.append(player_url)
    return player_urls

player_urls = get_player_urls("https://baseball-data.com/player/f/", headers)
player_urls

['https://baseball-data.com/player/f/%E3%83%9E%E3%83%AB%E3%83%86%E3%82%A3%E3%83%8D%E3%82%B9',
 'https://baseball-data.com/player/f/%E5%8A%A0%E8%97%A4%E3%80%80%E8%B1%AA%E5%B0%86',
 'https://baseball-data.com/player/f/%E4%B8%8A%E5%B7%9D%E7%95%91%E3%80%80%E5%A4%A7%E6%82%9F',
 'https://baseball-data.com/player/f/%E9%87%8E%E6%9D%91%E3%80%80%E4%BD%91%E5%B8%8C',
 'https://baseball-data.com/player/f/%E3%82%A2%E3%83%AB%E3%82%AB%E3%83%B3%E3%82%BF%E3%83%A9',
 'https://baseball-data.com/player/f/%E6%9D%BE%E6%9C%AC%E3%80%80%E5%89%9B',
 'https://baseball-data.com/player/f/%E6%B7%BA%E9%96%93%E3%80%80%E5%A4%A7%E5%9F%BA',
 'https://baseball-data.com/player/f/%E4%B8%AD%E5%B3%B6%E3%80%80%E5%8D%93%E4%B9%9F',
 'https://baseball-data.com/player/f/%E6%B8%85%E6%B0%B4%E3%80%80%E5%84%AA%E5%BF%83',
 'https://baseball-data.com/player/f/%E7%9F%A2%E6%BE%A4%E3%80%80%E5%AE%8F%E5%A4%AA',
 'https://baseball-data.com/player/f/%E7%94%9F%E7%94%B0%E7%9B%AE%E3%80%80%E7%BF%BC',
 'https://baseball-data.com/player/f/%E5%8A%A0%

In [5]:
# 各選手の個人データを取得する
def get_players_data(url, headers):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    uniform_number = soup.h2.text.split("(")[0].split(" ")[0]
    name = soup.h2.text.split("(")[0].split(" ")[1].replace("　", "")
    trs = soup.table.find_all("tr")
    all_data= []
    for tr in trs:
        tds = tr.find_all("td")
        for td in tds:
            all_data.append(td.text)
    born = all_data[0].replace("年", "/").replace("月", "/").replace("日", "/")
    height = all_data[6].replace("cm", "")
    weight = all_data[8].replace("kg", "")
    salary = all_data[9].replace("万円（推定）", "").replace(",", "")
    team_name = url.split("/")[-2]
    player_data = {
        "name": name,
        "born": born,
        "height": height,
        "weight": weight,
        "salary": salary,
        "uniform_number": uniform_number,
        "team_name": team_name,
    }

    return player_data

get_players_data("https://baseball-data.com/player/yb/%E6%88%B8%E6%9F%B1%E3%80%80%E6%81%AD%E5%AD%9D", headers)
# get_players_data("https://baseball-data.com/player/f/%E9%87%91%E6%9D%91%E3%80%80%E5%B0%9A%E7%9C%9F", headers)

{'name': '戸柱恭孝',
 'born': '1990/4/11/',
 'height': '179',
 'weight': '90',
 'salary': '4000',
 'uniform_number': '10',
 'team_name': 'yb'}